# To be run once to setup modules and paths


In [ ]:
#Reload modules without shutting notebook
%load_ext autoreload
%autoreload 2

import os
import shutil
import glob
from datetime import date
from IPython.display import clear_output
from ipywidgets import Dropdown, interact, interactive, fixed, interact_manual
import sys
sys.path.append('/usr/bin')
import gdal_merge as gm

# Do not show warnings
import warnings
warnings.filterwarnings('ignore')

# Global variables including folders
home = os.path.expanduser("~")
notebfolder = os.path.join(home, "notebooks")
basefolder = os.path.join(home, "my_shared_data_folder")
s1ardfolder = os.path.join(basefolder, "s1ard")
datasets = os.path.join(basefolder, "datasets")

# Upland burn modules
sys.path.append(os.path.join(notebfolder, "utils"))
import extract_aoi as extract
import get_configuration as getc
import portal_credentials as portalc
import call_cophub as callc
import onda_archive as onda
import syscmds as sysc

## setup temp folder
tmpfolder = os.path.join(home, "temp")
if not os.path.exists(tmpfolder):
    os.mkdir(tmpfolder)
tmptiffolder = os.path.join(tmpfolder, "tiffs")
if not os.path.exists(tmptiffolder):
    os.mkdir(tmptiffolder)

# Create a widget that can be used the choose the case study area of interest
list_of_cstudy = ["skye", "cairngorms", "pdistrict"]
cstudy_widget = Dropdown(options = list_of_cstudy, description = "Area:")
def change_cstudy(*args):
    print("Set to {}".format(cstudy_widget.value))
    
cstudy_widget.observe(change_cstudy, 'value')
print("Choose the case study area")
display(cstudy_widget)

In [ ]:
# Setup folders
basefolder, s1ardfolder, datasets, outfolder, tmpfolder, ofiles, hdfile, pfile, verbose, graphics = getc.get_configuration(cstudy)
if cstudy == "skye":
    aoi = os.path.join(datasets, "Skye_extent_OSGB36.geojson")
elif cstudy == "cairngorms":
    aoi = os.path.join(datasets, "Cairngorms_extent_OSGB36-extended.geojson")
else:
    aoi = os.path.join(datasets, "PDistrict_extent_OSGB36.geojson")
fstart = os.path.basename(aoi).split("_")[0]
aoifolder = os.path.join(basefolder, cstudy)

# Load Copernicus hub credentials
pfile = "cophub.txt"
home = os.path.expanduser("~")
if not os.path.exists(os.path.join(home, pfile)):
    portalc.save_credentials(pfile)
cop_credentials = portalc.read_credentials(pfile)

# Define date range and search Copernicus Hub

In [ ]:
# PDistrict - April to August 2018
# Cairngorms - March to mid June 2019
# Skye - Feb to April 2018
start_date = date(2018, 2, 1)
end_date = date(2018, 4, 30)

In [ ]:
# Call copernicus hub to get list of products
products, wkt = callc.call_cophub(cop_credentials, start_date, end_date, aoi, SLC = False)
print("Found {} products for {} to {} for {}".format(len(products),start_date,end_date,cstudy))

# Download Sentinel-1 ARD data for specific date range

In [ ]:
# Download ARD data
for download_num, (id1, data) in enumerate(products.iterrows()):
    filename = data['title']
    s1file = filename + '.tif'
    splits = filename.split("_")
    dstring = splits[4]
    dstring2 = splits[5]
    cedafile = splits[0] + "_" + dstring[0:8] + "_*_" + dstring[9:15] + "_" + dstring2[9:15] + "_*.tif"
    test = glob.glob(os.path.join(s1ardfolder, cedafile))
    # Check if already stored
    if (len(test) > 0) or os.path.exists(s1file):
        print("{} already downloaded".format(filename))
    else:
        # Load CEDA FTP credentials
        pfile = "ceda.txt"
        home = os.path.expanduser("~")
        if not os.path.exists(os.path.join(home, pfile)):
            portalc.save_credentials(pfile)
        ceda_credentials = portalc.read_credentials(pfile)

        print("Querying CEDA FTP")
        ceda = "ftp://ftp.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_1/"

        # Run curl command to try to download
        print("Trying to download {} ".format(filename))
        urlpath = ceda + os.path.join(dstring[0:4],os.path.join(dstring[4:6], dstring[6:8]))
        cmd = 'wget --user ' + ceda_credentials[0] + ' --password ' + ceda_credentials[1] + ' -P ' + s1ardfolder
        cmd += ' -r --no-directories --no-host-directories --no-parent -A "' + cedafile + '" ' + urlpath
        print("CMD: ",cmd)
        output = sysc.execmd(cmd, verb = False)
        if output != None:
            size = len(output)
            print(output[:size - 30]) # print last 30 characters


## Import the Sentinel-1 ARD data for the Area of Interest (AOI)

In [ ]:
# Import list of Sentinel-1 ARD (backscatter files)
s1ards = glob.glob(os.path.join(s1ardfolder, "*.tif")) # Edit to specify time range
numfiles  = len(s1ards)
print("Found {} Sentinel-1 ARD files".format(numfiles))

In [ ]:
# Subset each file to the area of interest
ofiles = []
print("Generating for {}".format(aoi))
if not os.path.exists(aoi):
    print("Could not find {}".format(aoi))
else:
    for ifile in s1ards:
        ofile = os.path.join(tmptiffolder, os.path.basename(ifile))
        
        # Skip if exists
        if os.path.exists(ofile):
            ofiles.append(ofile)
            print("{} exists, skipping".format(os.path.basename(ofile)))
        else:
            # Create subset in temp folder
            extract.cut_by_geojson(ifile, ofile, aoi, verb = True)

            if os.path.exists(ofile):
                ofiles.append(ofile)
                print("Generated: ",ofile)

print("Finished processing, {} output subsets available for {}".format(len(ofiles), cstudy))

# Check for same-day files and merge if they exist
## One generated the subset is manually moved to ~/my_shared_data_folder/datasets/[cstudy]/backscatter_tiffs

In [ ]:
# Sort files by names
ofiles = glob.glob(os.path.join(tmptiffolder, "*.tif")) # When running from pre-processed data in the temp directory
ofiles = sorted(ofiles,key=lambda x: int(os.path.splitext(os.path.basename(x))[0][4:12]))

subfiles = []
pdatestr = ""
for ofile in ofiles:
    dir, fname = os.path.split(ofile)
    fnames = fname.split("_")
    datestr = fnames[1]
    timestr = fnames[4]
    #print("Checking {} {} {}".format(fname, datestr, timestr))
    if datestr == pdatestr:
        merge = False
        dir, fname2 = os.path.split(pfile)
        if int(timestr) < int(ptimestr) and int(timestr[0:2]) == int(ptimestr[0:2]):
            mfile = os.path.join(dir, fname[:28] + fname2[28:])
            merge = True
        elif int(timestr) > int(ptimestr) and int(timestr[0:2]) == int(ptimestr[0:2]):
            mfile = os.path.join(dir, fname2[:28] + fname[28:])
            merge = True
        if merge and not os.path.exists(mfile) and os.path.exists(ofile) and os.path.exists(pfile):
            print("Merging {} and {} to create {}".format(pfile, ofile, mfile))
            shutil.copyfile(ofile,mfile)
            gm.main(['', '-init', '0', '-o', mfile, pfile, ofile])
            if os.path.exists(mfile):
                os.remove(pfile)
                os.remove(ofile)        
                subfiles.append(mfile)
            else:
                print("Failed to generate merged file {}".format(mfile))
        else:
            print("Merged file {} already exists".format(mfile))
            if os.path.exists(pfile):
                os.remove(pfile)
            if os.path.exists(ofile):
                os.remove(ofile)        
            subfiles.append(mfile)
            
    else:
        pdatestr = datestr
        ptimestr = timestr
        pfile = ofile
        subfiles.append(ofile)

# Create final list of files
ofiles = []
for file in subfiles:
    if os.path.exists(file):
        ofiles.append(file)
print("Finished processing, {} output subsets available for {}".format(len(ofiles), cstudy))